# Trust region policy optimization

- 높은 차원의 action space를 가진 환경부터 action은 적지만 control parameter가 많은 환경까지 범용적으로 사용할 수 있는 좋은 알고리즘이다.

## 서론

- 문제 정의, expected discounted reward를 최대화하는 policy를 찾자!

- MDP: \\(\langle S, A, P, r, \rho_0, \gamma \rangle\\)
  - \\(P \\): state transition probability
  - \\(\rho_0\\): 초기 state \\(s_0\\)에 대한 distribution
  - \\(r\\): reward function
  - \\(\gamma \\): discount factor

### Value based 강화학습

- Q-function으로 action을 선택한다.
- ex) SARSA, Q-Learning, DQN, Dueling network, prioritized experience replay, ...

### Policy based 강화학습

- policy를 가지고 있으며 policy는 parameter로 이루어져 있음.
- REINFORCE, Actor-critic, A3C, TRPO, PPO, GAE, ...

## Policy Gradient

- Objective function(policy parameter \\(\theta\\)의 함수)을 최대화하는 policy를 구하는 것
- 각각의 iteration마다 objective function의 gradient를 따라 paramter를 update한다.

### Policy gradient의 문제

- Sample의 효율성이 없다.
  - policy gradient는 현재 policy에 대한 estimate을 update하고 있는데, 한번 사용하면 버리게됨 따라서 효율성에서 떨어진다.
- parameter space에서 거리와 policy space에서 거리가 다르다.
  - parameter에서 update를 했을 때 policy가 받는 영향이 클수도 있고 작을 수도 있다.

### Importance sampling

- 효율적으로 기대값을 추정하기 위해 만들어졌다.
- 목적, 기대값을 계산하고자 하는 확률 분포 p(x)의 확률 밀도 함수를 알고 있지만 p에서 샘플을 생성하기 어려울 때 비교적 샘플을 생성하기 쉬운 q(x)에서 샘플을 생성하여 p의 기대값을 계산한다.

$$ E_{x\sim p}[f(x)] = \int p(x)f(x)dx = \int \frac{p(x)}{q(x)}q(x)f(x)dx = E_{x \sim q}\begin{bmatrix} \frac{p(x)}{q(x)}f(x) \end{bmatrix} $$

- 위 식에서 \\(\frac{p(x)}{q(x)}\\)를 likelihood ratio라 하며, \\(p\\)를 normal distribution, \\(q\\)를 importance distribution이라 한다.

- 따라서, importance sampling에서 \\(p\\)의 기대값 \\(E_{x \sim p}[f(x)]\\)를 아래와 같이 추정한다.
$$ E_{x \sim p}[f(x)] \approx \frac{1}{N} \sum_{n=1}^N \frac{p(x_n)}{q(x_n)}f(x_n),\; x_n\sim q $$

### policy gradient, importance sampling

- policy gradient에서 이전 policy의 sample을 사용하기 위해서 importance sampling 사용할 수 있음.
- importance weight(\\(\frac{p(x_n)}{q(x_n)}\\))는 unbound(variance의 범위가 커서) -> 학습이 잘 안된다.

### step in policy space

- 앞서 문제로 언급되었던, parameter space는 policy space가 아니다.
  - 그렇다면, parameter space가 아니라 policy space에서 조금씩 update가 가능한가?
    - 두 policy(old policy, new policy)사이에 KL-divergence를 constraint로 (trust region)

#### KL-divergence

- 얼마나 두 확률분포가 다른지를 설명해주는 지표
- \\(D_KL(P||Q) = \sum_i P(i)log \frac{P(i)}{Q(i)} \\)

- 주로 아직 확인되지 않은 모델을 특정 확률분포로 근사시키는데 사용한다.
- KL-divergence를 가장 낮추는 정규분포를 찾는다.

- Q(근사 모델)를 최대한 P(실제 데이터 분포)에 가깝게 만들려면, KL-divergence를 최소화하도록 학습을 시킨다.
- KL-divergence의 특징
  - 항상 0이상의 값을 갖는다.
  - 비대칭적 함수이다.
    - \\(D_KL(p||q)\\)와 \\(D_KL(q||p)\\)는 다른값이다.
  - 두 확률 분포가 동일하면 KL-divergence는 0이 된다.

- KL-divergence 분해하기
  $$ \begin{split} D_KL(P||Q) &= \sum_i P(i) log \frac{P(i)}{Q(i)} \\ &= \sum_i P(i)logP(i) - \sum_i P(i)logQ(i) \\ &= -H(P) + H(P,Q) \end{split} $$

- KL-divergence를 둘로 나눌 수 있기 때문에 크로스 엔트로피로 표현이 가능하다.

### stable한 update는 안되는가?

- lower bound를 최적화 하자!

### policy old와 policy new의 관계를 보자.

- policy \\(\pi_{old}\\)의 objective func, \\(J(\pi_{old}) = E_{s_0, a_0, ... \sim \pi_{old}}[\sum^\infty_{t=0} \gamma^t r(s_t)] \\)

- policy \\(\pi\\) objective func, \\(J(\pi) = E_{s_0, a_0, ... \sim \pi}[\sum^\infty_{t=0} \gamma^t r(s_t)] \\)

- \\(J(\pi) = J(\pi_{old}) + E_{s_0, a_0, ... \sim \pi} \begin{bmatrix} \sum^\infty_{t=0} \gamma A_{\pi_{old}}(s_t, a_t) \end{bmatrix} \\)
<br><br>
- \\(J(\pi) = J(\pi_{old}) + \sum_s \rho_\pi(s) \sum_a \pi(a|s) A_{\pi_{old}}(s,a)\\)

### policy Iteration & objective function

- policy improvement -> greedy policy improvement
  - 어떤 행동 상태가 positive advantage를 가지면 policy를 improve한다.
  - \\( \pi = argmax_a A_{\pi_{old}}(s,a) \\)
- approximation error -> \\(A_{\pi_{old}}(s,a) < 0 \\)가능, \\( \rho_\pi(s) \\)구하기 어렵다.

- \\(J(\pi)\\) -> local approximation
$$ L_{\pi_{old}}(\pi) = J(\pi_{old}) + \sum_s \rho_{\pi_{old}}(s) \sum_a \pi(a|s) A_{\pi_{old}}(s,a) $$

### Local Approximation

- \\(J(\pi) = J(\pi_{old}) + \sum_s {\color{Red}{\rho_\pi(s)}} \sum_a \pi(a|s) A_{\pi_{old}}(s,a)\\)
<br><br>
- 위 식에서 아래 식으로 빨간색 부분이 바뀌었는데, 이것은 policy의 변화가 적다면 steady state distribution의 변화도 무시할 수 있을 것이다.
- \\( L_{\pi_{old}}(\pi) = J(\pi_{old}) + \sum_s {\color{Red} {\rho_{\pi_{old}}(s)}} \sum_a \pi(a|s) A_{\pi_{old}}(s,a) \\)

- 그렇다면, Approximation error는 얼마나 되냐? -> \\(\color{Red}{\text{Lower bound}}\\)

### Local approximation with parameterized policy

- 위 식은,
- \\(L_{\pi_{\theta_0}}(\pi_{\theta_0}) = J(\pi_{\theta_0})\\)으로 표현 가능하고,
- Kakade & Langford의 증명, 따라서 \\(\pi_{\theta_0}\\)에서 policy를 조금만 변화시켰을 때 local approximation을 improve -> objective function도 improve함.

### Conservative policy iteration

- 위에서 local approximation을 improve하는 것이 objective function도 improve하는 것을 증명 했지만, 어느 정도 변해야 objective function의 improve를 보장하는지 알 수 없다.
- 따라서, explicit한 J의 lower bound를 제시한다.

1. policy improvement(mixture policy 가정)
  - 현재 policy: \\(\pi_{old}\\), 새로운 policy: \\(\pi_{new}\\)
  - \\(\pi' = argmax_{\pi'}L_{\pi_{old}}(\pi')\\)
  - \\(\pi_{new}(a|s) = (1-\alpha)\pi_{old}(a|s) + \alpha \pi'(a|s)\\)

2. Lower bound of J
  - \\(J(\pi_{new}) \geq L_{\pi_{old}}(\pi_{new}) - \frac{2\epsilon \gamma}{(1-\gamma)^2} \alpha^2 \\)
  - \\(where \epsilon = max_s|E_{a \sim \pi'(a|s)}[A_\pi(s,a)]| \\)
  - 따라서 이제는 lower bound를 최적화하면 된다.
  - parameterized policy에 대해서 mixture policy를 구할 수 없기 때문.

### Conservative policy Iteration의 변형

- mixture policy를 통해 old policy와 new policy사이의 거리를 나타내지 않고, KL-divergence를 사용한다.
  - \\( \alpha^2 -> D^{max}_{KL}(\pi_{old}, \pi_{new}) \\)
- 변형된 lower bound 식 사용
  - \\( \eta(\pi_{new}) \geq L_{\pi_{old}}(\pi_{new}) - CD^{max}_{KL}(\pi_{old}, \pi_{new})\\)
  - \\( where C = \frac{4 \epsilon \gamma}{(1 - \gamma)^2}, \epsilon=max_s|E_{a \sim \pi'(a|s)}[A_\pi(s,a)]| \\)

### Lower bound of objective function

- \\(J(\pi_{new}) \geq L_{\pi_{old}}(\pi_{new}) - CD^{max}_{KL}(\pi_{old}, \pi_{new}) \\)
- 현재 \\(\theta\\)에서는 값이 모두 같다.

![default](https://user-images.githubusercontent.com/22078438/49554137-3bb7f400-f93e-11e8-891a-6cb11a27d9c1.PNG)


### KL-constraint optimization

- parameterized policy를 lower bound식에 적용한다.
  - \\(J(\theta) \geq L_{\theta_{old}}(\theta) - CD^{max}_{KL}(\theta_{old}, \theta) \\)
- Lower bound의 optimization: C가 너무 커서 step이 작다.
  - \\(maximize_{\theta}[L_{\theta_{old}}(\theta) - CD^{max}_{KL}(\theta_{old}, \theta)] \\)
- Large and robust step: KL-penalty -> KL-constraint
  - \\(maximize_\theta L_{\theta_{old}}(\theta) \\)
  - \\(s. t. D^{max}_{KL}(\theta_{old}, \theta) \leq \delta\\)

### 정보의 양

- 위 KL-constraint를 정확하게 알기 위해서 entropy의 개념과 KL-divergence의 개념을 다시 공부해봅니다.

- 랜덤하게 나타나는 숫자가 x이고 그 숫자 x가 나타날 확률이 \\(p(x)\\)라 하면 확률이 낮은 정보일수록 희소성이 있다.
- 정보의 양, \\(h(x) = -logp(x)\\)
  - 로그 함수의 성질에 따라 \\(p(x)\\)가 0에 가까워지면 무한대로 가고,
  - 1일때는 0이다. 따라서 \\(p(1)\\)은 항상 일어나는 일이라는 의미가 된다.

- entropy
  - 여러 사건이 발생할 경우 entropy는 \\(h(x)\\)의 weighted sum으로 정의한다.
  - \\(H[x] = -\sum^N_{i=1} p_i logp_i\\)
  - 위 식은 사건이 많아도 쉽게 계산이 되는 특징이 있고, \\(p(x)\\)들의 합은 무조건 1이 되어야한다.

### KL-constraint optimization

- \\(maximize_\theta L_{\theta_{old}}(\theta) \\)<br><br>
- \\(s.t. D^{max}_{KL}(\theta_{old}, \theta) \leq \delta\\)
  - policy space에서는 작은 step이다.(??)

![default](https://user-images.githubusercontent.com/22078438/49581154-5cab3400-f994-11e8-9a35-1a53ca938fdb.PNG)


- 위 그림에서 \\(\eta(\theta)\\)는 최대화하고 싶은 expected discounted rewards이다.
- \\(L(\theta)-C \cdot KL\\)은 lower bound function이다.

### Surrogate advantage function

- \\(L_{\theta_{old}}(\theta) = J(\theta_{old}) + \sum_s \rho_{\pi_{\theta_{old}}}(s) \sum_a \pi_{\theta}(a|s) A_{\pi_{\theta_{old}}}(s, a)\\)

- \\(maximize_{\theta} L_{\theta_{old}}(\theta) \rightarrow maximize_{\theta} \begin{bmatrix} \sum_s \rho_{\pi_{\theta_{old}}}(s) \sum_a \pi_\theta(a|s) A_{\pi_{\theta_{old}}}(s, a) \end{bmatrix} \\)

- 그렇다면, \\(\sum_s \rho_{\pi_{\theta_{old}}}(s) \sum_a \pi_\theta(a|s) A_{\pi_{\theta_{old}}}(s, a)\\) 이부분을 어떻게 구하냐? -> approximation

- \\(\sum_s \rho_{\pi_{\theta_{old}}}(s) \sum_a \pi_\theta(a|s) A_{\pi_{\theta_{old}}}(s, a) = E_{s \sim \pi_{\theta_{old}}, a \sim \pi_\theta} \begin{bmatrix} A_{\pi_{\theta_{old}}}(s,a) \end{bmatrix} \\)

- \\( E_{s \sim \pi_{\theta_{old}}, a \sim \pi_\theta} \begin{bmatrix} \frac{\pi_\theta(a|s)}{\pi_{\theta_{old}}(a|s)} A_{\pi_{\theta_{old}}}(s,a) \end{bmatrix} \\)
  - importance sampling 사용
  - 위 식이 surrogate advantage function

- 결과적으로,
  - \\(maximize_\theta ( E_{s \sim \pi_{\theta_{old}}, a \sim \pi_\theta} \begin{bmatrix} \frac{\pi_\theta(a|s)}{\pi_{\theta_{old}}(a|s)} A_{\pi_{\theta_{old}}}(s,a) \end{bmatrix} \; s.t. \overline{D^{\rho_{old}}_{KL}}(\theta_{old}, \theta) \leq \delta \\)

## TRPO를 공부하는데 참조하였습니다.

[가깝고도 먼 Trpo](https://www.slideshare.net/WoongwonLee/trpo-87165690)